In [210]:
import os
import time
import requests
import pyspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, when, round, desc

# Create Spark session
spark = SparkSession.builder.appName("Global_Energy_Analysis").master("local[1]").getOrCreate()

# Download OWID dataset from GitHub
url = "https://raw.githubusercontent.com/owid/energy-data/master/owid-energy-data.csv"
local_path = os.path.abspath("owid-energy-data.csv")
r = requests.get(url)
r.raise_for_status()
with open(local_path, "wb") as f:
    f.write(r.content)

# Load ALL data 
df = spark.read.option("header", True).option("inferSchema", True).csv(local_path)

#df remove all tings that are not countries
df = df.filter(col("iso_code").rlike("^[A-Z]{3}$") & (col("year") >= 2000))

print("Setup complete!")

Setup complete!


In [211]:
#Query 1 - Global Electricity Consumption (2000-2023)
print("\n" + "="*60)
print("QUERY 1: Global Electricity Consumption (2000-2023)")
print("="*60)

start_time = time.time()

# Aggregate electricity generation globally by year
query1 = df.select("year", "electricity_generation") \
    .filter(col("electricity_generation").isNotNull()) \
    .groupBy("year") \
    .agg(sum("electricity_generation").alias("total_electricity_generation")) \
    .orderBy("year")

# Calculate total growth and average annual growth
first_year_value = query1.filter(col("year") == 2000).select("total_electricity_generation").first()[0]
last_year_value = query1.filter(col("year") == 2023).select("total_electricity_generation").first()[0]
total_growth = ((last_year_value - first_year_value) / first_year_value) * 100

query_time = time.time() - start_time

# Display results
query1.show(50, truncate=False)  
print(f"\nKEY INSIGHTS:")
print(f"- World electricity generation grew from {first_year_value:.0f} TWh to {last_year_value:.0f} TWh")
print(f"- Total growth: {total_growth:.1f}%")
print(f"- Average annual growth: {(total_growth/23):.1f}%")
print(f"- Query execution time: {query_time:.2f} seconds")


QUERY 1: Global Electricity Consumption (2000-2023)
+----+----------------------------+
|year|total_electricity_generation|
+----+----------------------------+
|2000|15275.730000000009          |
|2001|15496.690000000002          |
|2002|16045.900000000003          |
|2003|16624.100000000002          |
|2004|17410.58000000001           |
|2005|18129.379999999997          |
|2006|18834.580000000005          |
|2007|19709.119999999995          |
|2008|20097.310000000005          |
|2009|19938.47                    |
|2010|21258.349999999995          |
|2011|21966.02                    |
|2012|22509.679999999997          |
|2013|23148.63                    |
|2014|23741.6                     |
|2015|23996.98999999998           |
|2016|24689.249999999993          |
|2017|25435.96000000001           |
|2018|26459.289999999994          |
|2019|26834.069999999985          |
|2020|26722.319999999985          |
|2021|28252.730000000007          |
|2022|28911.240000000005          |
|2023|29541

In [212]:
#Query 2 - Global Renewable Energy Growth (2000-2023)
print("\n" + "="*60)
print("QUERY 2: Global Renewable Energy Growth (2000-2023)")
print("="*60)

start_time = time.time()

# Aggregate renewable electricity globally by year
query2 = df.select("year", 
                   "renewables_electricity",
                   "electricity_generation") \
    .filter(col("electricity_generation").isNotNull()) \
    .groupBy("year") \
    .agg(
        sum("renewables_electricity").alias("total_renewables_electricity"),
        sum("electricity_generation").alias("total_electricity_generation")
    ) \
    .withColumn("renewable_share", 
                round((col("total_renewables_electricity") / col("total_electricity_generation")) * 100, 2)) \
    .select("year", "total_renewables_electricity", "total_electricity_generation", "renewable_share") \
    .orderBy("year")

# Get values for calculations
renewable_2000 = query2.filter(col("year") == 2000).select("renewable_share").first()[0]
renewable_2023 = query2.filter(col("year") == 2023).select("renewable_share").first()[0]
renewable_growth_pct = renewable_2023 - renewable_2000

renewable_twh_2000 = query2.filter(col("year") == 2000).select("total_renewables_electricity").first()[0]
renewable_twh_2023 = query2.filter(col("year") == 2023).select("total_renewables_electricity").first()[0]
renewable_abs_growth = renewable_twh_2023 - renewable_twh_2000
renewable_pct_growth = ((renewable_twh_2023 - renewable_twh_2000) / renewable_twh_2000) * 100

query_time = time.time() - start_time

query2.show(50, truncate=False)
print(f"\nKEY INSIGHTS:")
print(f"- Global renewable electricity grew from {renewable_twh_2000:.0f} TWh to {renewable_twh_2023:.0f} TWh")
print(f"- Total growth: {renewable_pct_growth:.1f}%")
print(f"- Average annual growth: {(renewable_pct_growth/23):.1f}%")
print(f"- Renewable share increase: {renewable_growth_pct:.1f} percentage points (from {renewable_2000}% to {renewable_2023}%)")
print(f"- Query execution time: {query_time:.2f} seconds")


QUERY 2: Global Renewable Energy Growth (2000-2023)
+----+----------------------------+----------------------------+---------------+
|year|total_renewables_electricity|total_electricity_generation|renewable_share|
+----+----------------------------+----------------------------+---------------+
|2000|2859.8399999999997          |15275.730000000009          |18.72          |
|2001|2797.03                     |15496.690000000002          |18.05          |
|2002|2874.1800000000003          |16045.900000000003          |17.91          |
|2003|2900.26                     |16624.100000000002          |17.45          |
|2004|3131.4800000000005          |17410.58000000001           |17.99          |
|2005|3281.369999999998           |18129.379999999997          |18.1           |
|2006|3433.9600000000005          |18834.580000000005          |18.23          |
|2007|3543.100000000001           |19709.119999999995          |17.98          |
|2008|3800.77                     |20097.310000000005   

In [213]:
#Query 3 - Global Renewable Electricity Leaders (2023)
print("\n" + "="*60)
print("QUERY 3: Global Renewable Electricity Leaders (2023)")
print("="*60)

start_time = time.time()

# Filter for 2023 and calculate renewable share for each country
query3 = df.filter((col("year") == 2023)) \
    .select("country", 
            "renewables_electricity",
            "electricity_generation") \
    .filter(col("electricity_generation").isNotNull()) \
    .withColumn("renewable_share", 
                round((col("renewables_electricity") / col("electricity_generation")) * 100, 2)) \
    .select("country", "renewable_share") \
    .orderBy(desc("renewable_share"))

# Calculate global average renewable share
global_avg = query3.agg({"renewable_share": "avg"}).collect()[0][0]

query_time = time.time() - start_time

query3.show(50, truncate=False)
print(f"\nKEY INSIGHTS (Year: 2023):")
print(f"- Global average renewable share: {global_avg:.1f}%")
print(f"- Query execution time: {query_time:.2f} seconds")

# Stop Spark session
spark.stop()


QUERY 3: Global Renewable Electricity Leaders (2023)
+----------------------------+---------------+
|country                     |renewable_share|
+----------------------------+---------------+
|Albania                     |100.0          |
|Bhutan                      |100.0          |
|Democratic Republic of Congo|100.0          |
|Ethiopia                    |100.0          |
|Iceland                     |100.0          |
|Paraguay                    |100.0          |
|Costa Rica                  |99.91          |
|Norway                      |98.47          |
|Namibia                     |97.88          |
|Eswatini                    |96.43          |
|Sierra Leone                |95.24          |
|Tajikistan                  |92.59          |
|Uruguay                     |92.0           |
|El Salvador                 |90.62          |
|Kenya                       |89.84          |
|Luxembourg                  |89.29          |
|Zambia                      |89.0           |
|Brazi